# **[ Project 05. Building Image Classification Model : RockScissorPaper Image Classification with Tensorflow ]**


In [ ]:
##############################################  구글드라이브 마운트 #####################################################
from google.colab import drive
import os

# 구글드라이브 마운트 
root = "/content/drive"
drive.mount(root, force_remount=True)



Mounted at /content/drive


In [ ]:

################### 데이터프레임으로 이미지 불러오고 크기 조정 ##################3

import pandas as pd
import os
import glob
from PIL import Image
import numpy as np 
from pathlib import Path


# 데이터 경로 지정 
googledrive_path = "MyDrive/github/ML-DL-Project"
project_dir = "Project_05_Building Image Classification Model : RockScissorPaper Image Classification with Tensorflow"
data_path = os.path.join(root,googledrive_path,project_dir,"data")

class ImageProcessing:

  def __init__(self,img_path):
    self.img_path = img_path

  def define_image_path(self):
    img_extension_list = [ '.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP']
    self.img_path_list = [data for data in Path(self.img_path).glob('**/*') if data.suffix in img_extension_list] #이미지파일 찾기
    return self.img_path_list


  def resize_image(self,target_width,target_height):
    for img_path in self.img_path_list:
      
      img = Image.open((img_path))
      img = img.resize((int(target_width), int(target_height)))
      new_root_folder = str(img_path.parent.parent) + "_resize_"+str(target_width)+"x"+str(target_height)
      new_folder = new_root_folder+"/"+str(img_path.parts[-2])
      img_name = img_path.parts[-1]
      resize_file = new_folder +"/" + img_name
      if not os.path.exists(new_folder):
        # Path(new_folder).mkdir()
        Path(new_folder).mkdir(parents=True, exist_ok=True)
        print(new_folder, " 생성완료")


      if not os.path.exists(resize_file):
        img.save(resize_file)

    print("resize 완료")
    img_extension_list = [ '.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP']
    self.img_path_list = [data for data in Path(new_root_folder).glob('**/*') if data.suffix in img_extension_list]


  def set_image_label(self,label_dict=None):
    if  label_dict == None:
      label_dict = dict([(value.parts[-1],idx) for idx, value in enumerate(Path(data_path).glob('*/')) ])
    self.label_list = [label_dict[img_path.parts[-2]] for img_path in self.img_path_list] 
    return self.label_list

  
  def load_dataset(self,target_width,target_height):
    number_of_data = len(self.img_path_list)
    # img_array_list = [np.array(Image.open(img_path),dtype=np.int32) for img_path in self.img_path_list ]
    imgs=np.zeros(number_of_data*target_width*target_height*3,dtype=np.int32).reshape(number_of_data,target_width,target_height,3)
    
    idx = 0
    for img_path in self.img_path_list:
      img = np.array(Image.open(img_path).convert("RGB"),dtype=np.int32)
      # print(img.mode)
      imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
      idx=idx+1
    labels = np.array(self.set_image_label())
    return imgs,labels

      
    print("--")
    print('class to index: ',label_dict)
    print('label_list: ',label_list)
    print('img_array_list: ',img_array_list)


In [ ]:
a = ImageProcessing(data_path)
img_path_list = a.define_image_path()

a.resize_image(28,28)

X, y = a.load_dataset(28,28)


X_norm = X/255.0   # 입력은 0~1 사이의 값으로 정규화


/content/drive/MyDrive/github/ML-DL-Project/Project_05_Building Image Classification Model : RockScissorPaper Image Classification with Tensorflow/data_resize_28x28/rock  생성완료
/content/drive/MyDrive/github/ML-DL-Project/Project_05_Building Image Classification Model : RockScissorPaper Image Classification with Tensorflow/data_resize_28x28/scissor  생성완료
/content/drive/MyDrive/github/ML-DL-Project/Project_05_Building Image Classification Model : RockScissorPaper Image Classification with Tensorflow/data_resize_28x28/paper  생성완료
resize 완료


In [ ]:

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3,  random_state=1004)
X_train_scaled,X_test_scaled,y_train,y_test = train_test_split(X_norm, y, test_size=0.3,  random_state=1004)




In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


model = keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (5,5), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(4,4))
model.add(keras.layers.Conv2D(32,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(15, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

print('Model에 추가된 Layer 개수: ', len(model.layers))
model.summary()


Model에 추가된 Layer 개수:  8
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 24, 24, 16)        1216      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 6, 6, 16)         0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 4, 4, 32)          4640      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 2, 2, 32)         0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 128)               0         
                                                                 
 dense_12 (Dense)            (

In [ ]:
model.compile(optimizer='adam',
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy']    )

print("====== 정규화 x ============")
model.fit(X_train,y_train, epochs=10)
print("====== 정규화 o ============")
model.fit(X_train_scaled,y_train, epochs=10)

====== 정규화 x ============
Epoch 1/10
62/62 [==============================] - 2s 15ms/step - loss: 1.6009 - accuracy: 0.8090
Epoch 2/10
62/62 [==============================] - 1s 15ms/step - loss: 0.1961 - accuracy: 0.9458
Epoch 3/10
62/62 [==============================] - 1s 15ms/step - loss: 0.0771 - accuracy: 0.9772
Epoch 4/10
62/62 [==============================] - 1s 15ms/step - loss: 0.0580 - accuracy: 0.9833
Epoch 5/10
62/62 [==============================] - 1s 15ms/step - loss: 0.0812 - accuracy: 0.9772
Epoch 6/10
62/62 [==============================] - 1s 15ms/step - loss: 0.0871 - accuracy: 0.9721
Epoch 7/10
62/62 [==============================] - 1s 15ms/step - loss: 0.0732 - accuracy: 0.9772
Epoch 8/10
62/62 [==============================] - 1s 15ms/step - loss: 0.0411 - accuracy: 0.9873
Epoch 9/10
62/62 [==============================] - 1s 15ms/step - loss: 0.0224 - accuracy: 0.9914
Epoch 10/10
62/62 [==============================] - 1s 15ms/step - loss: 0.0080 - 

In [ ]:
print("====== 정규화 x ============")
test_loss, test_accuracy = model.evaluate(X_test,y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))


print("====== 정규화 o ============")
test_loss, test_accuracy = model.evaluate(X_test_scaled,y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

====== 정규화 x ============
27/27 - 0s - loss: 154.1409 - accuracy: 0.7237 - 151ms/epoch - 6ms/step
test_loss: 154.1409454345703 
test_accuracy: 0.7237308025360107
====== 정규화 o ============
27/27 - 0s - loss: 0.0607 - accuracy: 0.9870 - 160ms/epoch - 6ms/step
test_loss: 0.06066831201314926 
test_accuracy: 0.9870129823684692
